In [10]:
# 📦 Install required packages (this takes a few minutes)
!pip install flask-ngrok pyngrok transformers torch torchaudio sentence-transformers TTS librosa soundfile inflect

In [11]:


# 📂 Imports
from flask import Flask, request, jsonify, send_file
from flask_ngrok import run_with_ngrok
import torch, os
import torchaudio
from transformers import AutoModelForCausalLM, AutoTokenizer, Wav2Vec2Processor, Wav2Vec2ForCTC
from sentence_transformers import SentenceTransformer, util
from TTS.api import TTS

# 📁 Create upload folder
os.makedirs("uploads", exist_ok=True)


In [12]:
!pip install TTS==0.17.3 torch torchaudio --upgrade --quiet
!pip install librosa soundfile scipy inflect


ERROR: Ignored the following versions that require a different python version: 0.0.10.2 Requires-Python >=3.6.0, <3.9; 0.0.10.3 Requires-Python >=3.6.0, <3.9; 0.0.11 Requires-Python >=3.6.0, <3.9; 0.0.12 Requires-Python >=3.6.0, <3.9; 0.0.13.1 Requires-Python >=3.6.0, <3.9; 0.0.13.2 Requires-Python >=3.6.0, <3.9; 0.0.14.1 Requires-Python >=3.6.0, <3.9; 0.0.15 Requires-Python >=3.6.0, <3.9; 0.0.15.1 Requires-Python >=3.6.0, <3.9; 0.0.9 Requires-Python >=3.6.0, <3.9; 0.0.9.1 Requires-Python >=3.6.0, <3.9; 0.0.9.2 Requires-Python >=3.6.0, <3.9; 0.0.9a10 Requires-Python >=3.6.0, <3.9; 0.0.9a9 Requires-Python >=3.6.0, <3.9; 0.1.0 Requires-Python >=3.6.0, <3.10; 0.1.1 Requires-Python >=3.6.0, <3.10; 0.1.2 Requires-Python >=3.6.0, <3.10; 0.1.3 Requires-Python >=3.6.0, <3.10; 0.10.0 Requires-Python >=3.7.0, <3.11; 0.10.1 Requires-Python >=3.7.0, <3.11; 0.10.2 Requires-Python >=3.7.0, <3.11; 0.11.0 Requires-Python >=3.7.0, <3.11; 0.11.1 Requires-Python >=3.7.0, <3.11; 0.12.0 Requires-Python >=3

In [13]:
!apt-get install -y espeak


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
espeak is already the newest version (1.48.15+dfsg-3).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [14]:
from TTS.api import TTS

# Load the model safely
try:
    tts = TTS(model_name="tts_models/en/ljspeech/vits", progress_bar=False)
    tts.to("cpu")  # use "cuda" if you want GPU
    print("✅ VITS model loaded successfully!")
except Exception as e:
    print("❌ Failed to load TTS model:", e)


 > tts_models/en/ljspeech/vits is already downloaded.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
✅ VITS model loaded successfully!


In [15]:
# GPT-2 for sentence generation
gpt_tokenizer = AutoTokenizer.from_pretrained("gpt2")
gpt_model = AutoModelForCausalLM.from_pretrained("gpt2")

# Wav2Vec2.0 for transcription
wav2vec_processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")
wav2vec_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")

# BERT model for scoring
bert_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# TTS (VITS)
# tts = TTS(model_name="tts_models/en/ljspeech/vits", progress_bar=False)
# tts.to("cpu")


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
from pyngrok import ngrok

# Replace with your actual authtoken
ngrok.set_auth_token("2vqKGjhbEbTD5U4SxESLltZxaLp_85iX2dgDarFqRV7VRVkVE")


In [17]:
!pip install flask-cors


In [23]:
from google.colab import files
uploaded = files.upload()


Saving scriptAR.js to scriptAR.js


In [19]:
!apt-get install ffmpeg -y


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [20]:
import subprocess


In [21]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import random


In [ ]:
#  Setup
from flask_cors import CORS
app = Flask(__name__)
CORS(app)
os.makedirs("uploads", exist_ok=True)


# Start ngrok tunnel
port = 5000
public_url = ngrok.connect(port)
print(f"🚀 Public URL: {public_url}")

#  Serve frontend files
@app.route("/")
def serve_index():
    return send_file("indexAR.html")

@app.route("/script.js")
def serve_script():
    return send_file("scriptAR.js")

# 📌 GET random sentence
@app.route("/generate_sentence")
def generate_sentence():
    input_text = "A dog is sitting"
    input_ids = gpt_tokenizer.encode(input_text, return_tensors="pt")

    output = gpt_model.generate(
        input_ids,
        max_length=15,
        num_return_sequences=1,
        do_sample=True,
        top_p=0.9,
        temperature=0.7,
        eos_token_id=gpt_tokenizer.eos_token_id,
        early_stopping=True
    )

    generated_text = gpt_tokenizer.decode(output[0], skip_special_tokens=True)
    sentences = generated_text.split('.')
    sentence = sentences[0].strip() + '.'

    return jsonify({"sentence": sentence})


# 📌 POST audio + evaluate
@app.route("/evaluate_speech", methods=["POST"])
def evaluate_speech():
    original_text = request.form.get("original")
    audio = request.files["audio"]
    webm_path = "uploads/user_audio.webm"
    wav_path = "uploads/user_audio.wav"
    audio.save(webm_path)

    # 🔄 Convert webm → wav using ffmpeg
    subprocess.run([
        "ffmpeg", "-i", webm_path, "-ar", "16000", "-ac", "1", wav_path, "-y"
    ], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # Load the wav file for transcription
    waveform, sr = torchaudio.load(wav_path)
    if sr != 16000:
        waveform = torchaudio.transforms.Resample(sr, 16000)(waveform)

    input_values = wav2vec_processor(waveform.squeeze(), return_tensors="pt", sampling_rate=16000).input_values
    with torch.no_grad():
        logits = wav2vec_model(input_values).logits
    ids = torch.argmax(logits, dim=-1)
    spoken_text = wav2vec_processor.batch_decode(ids)[0]

    # Compare similarity with BERT
    embeddings = bert_model.encode([original_text, spoken_text], convert_to_tensor=True)
    score = util.pytorch_cos_sim(embeddings[0], embeddings[1]).item() * 100
    feedback = "Excellent pronunciation!" if score >= 95 else "Good, but improve fluency." if score >= 85 else "Work on pronunciation clarity.Try Again!"

    return jsonify({
        "original": original_text,
        "transcribed": spoken_text.strip(),
        "score": round(score, 2),
        "feedback": feedback
    })

# 📌 POST TTS
@app.route("/generate_audio", methods=["POST"])
def generate_audio():
    sentence = request.json.get("sentence")
    output_path = "uploads/output.wav"
    tts.tts_to_file(text=sentence, file_path=output_path)
    return send_file(output_path, mimetype="audio/wav")

# ▶️ Start the server
app.run(port=5000)

🚀 Public URL: NgrokTunnel: "https://f8e2-34-45-31-228.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [23/Apr/2025 09:14:37] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Apr/2025 09:14:38] "GET /script.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Apr/2025 09:14:47] "GET /favicon.ico HTTP/1.1" 404 -
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
INFO:werkzeug:127.0.0.1 - - [23/Apr/2025 09:15:05] "GET /generate_sentence HTTP/1.1" 200 -
